# Trans-pQTL Colocalization

**Created**: 26 March 2022

## Environment

In [ ]:
if (!requireNamespace("patchwork")) {
    install.packages("devtools")
    devtools::install_github("thomasp85/patchwork")
}

In [ ]:
library(tidyverse)
library(data.table)
library(coloc)
library(patchwork)

setwd("~/eQTL_pQTL_Characterization/")

source("01_Colocalization/scripts/utils/ggplot_theme.R")

## Load Summary Information

In [ ]:
cis.pqtl.summary <- readRDS("/nfs/users/nfs_n/nm18/gains_team282/proteomics/pqtl/pqtl_ms2019/cis_pqtl_all.RDS") %>%
    as.data.frame() %>%
    dplyr::mutate(pQTL_ID=pQTL_Protein)

In [ ]:
head(cis.pqtl.summary)

In [ ]:
cis.pqtl.loci <- split(cis.pqtl.summary, cis.pqtl.summary$pQTL_ID)

In [ ]:
trans.pqtl.summary <- readRDS("/nfs/users/nfs_n/nm18/gains_team282/proteomics/pqtl/pqtl_ms2019/trans_pqtl_all.RDS") %>%
    as.data.frame() %>%
    dplyr::mutate(pQTL_ID=paste0(pQTL_Protein, "-", pQTL_Locus))

In [ ]:
head(trans.pqtl.summary)

Split summary statistics by locus.

In [ ]:
trans.pqtl.loci <- split(trans.pqtl.summary, trans.pqtl.summary$pQTL_ID)

## Load Protein Expression

In [ ]:
gene.exp <- read.table("/lustre/scratch119/humgen/projects/gains_team282/eqtl/data/logcpm_864_20412_hla.txt")
gene.exp <- t(gene.exp)
rownames(gene.exp) <- gsub("^GA", "", rownames(gene.exp))

In [ ]:
head(gene.exp)

In [ ]:
pqtl.geno <- fread("/nfs/users/nfs_n/nm18/gains_team282/Genotyping/All_genotyping_merged_filtered_b38_refiltered_rsID.fam", drop=3:6)
colnames(pqtl.geno) <- c("family", "individual")
protein.exp <- read.csv("/lustre/scratch119/humgen/projects/gains_team282/proteomics/MS2019_processed_data/data_291x1860_MS2019.csv", row.names=1)
protein.exp <- t(protein.exp)
rownames(protein.exp) <- gsub("^X", "", gsub("\\..*$", "", rownames(protein.exp)))
protein.info <- read.csv("/lustre/scratch119/humgen/projects/gains_team282/proteomics/MS2019_processed_data/sample_info_1860_MS2019.csv", row.names=1)
protein.info <- protein.info %>%
    dplyr::filter(Patient %in% pqtl.geno$individual)
protein.info$Patient <- gsub("^GA", "", protein.info$Patient)
protein.exp <- protein.exp[paste0(protein.info$Injection),]

In [ ]:
head(protein.exp)

## COLOC

In [ ]:
trans.pqtl.geno <- fread("~/gains_team282/nikhil/data/genotypes/trans_pqtl_genotypes.raw", drop=2:6, sep=" ") %>%
    as.data.frame()

rownames(trans.pqtl.geno) <- trans.pqtl.geno$FID
trans.pqtl.geno$FID <- NULL
colnames(trans.pqtl.geno) <- gsub("_.*$", "", colnames(trans.pqtl.geno))
trans.pqtl.geno <- as.matrix(trans.pqtl.geno)

In [ ]:
trans.pqtl.summary %>%
    dplyr::group_by(pQTL_ID) %>%
    dplyr::summarize(Chr=Chr[1])

In [ ]:
chr.14.ids <- c("P00751-pQTL_1", "P02787-pQTL_1", "Q92954-pQTL_1")

chr.14.loci <- lapply(chr.14.ids, function(id) {

    locus <- list()
    locus$beta <- trans.pqtl.loci[[id]]$pQTL_beta
    locus$varbeta <- trans.pqtl.loci[[id]]$pQTL_SE^2
    locus$snp <- trans.pqtl.loci[[id]]$SNP
    locus$position <- trans.pqtl.loci[[id]]$Position
    locus$type <- "quant"
    locus$sdY <- sd(protein.exp[,trans.pqtl.loci[[id]]$pQTL_Protein[1]], na.rm=TRUE)
    locus$LD <- cor(trans.pqtl.geno[, trans.pqtl.loci[[id]]$SNP], use="pairwise.complete.obs")
    
    locus.S = runsusie(locus)
    
    return(locus.S)
})

In [ ]:
susie.res = suppressMessages(coloc.susie(chr.14.loci[[1]], chr.14.loci[[2]]))

susie.res.df.1 = data.frame(susie.res$summary)
susie.res.df.1$trans.pQTL.1 = chr.14.ids[1]
susie.res.df.1$trans.pQTL.2 = chr.14.ids[2]

In [ ]:
susie.res = suppressMessages(coloc.susie(chr.14.loci[[2]], chr.14.loci[[3]]))

susie.res.df.2 = data.frame(susie.res$summary)
susie.res.df.2$trans.pQTL.1 = chr.14.ids[2]
susie.res.df.2$trans.pQTL.2 = chr.14.ids[3]

In [ ]:
susie.res = suppressMessages(coloc.susie(chr.14.loci[[1]], chr.14.loci[[3]]))

susie.res.df.3 = data.frame(susie.res$summary)
susie.res.df.3$trans.pQTL.1 = chr.14.ids[1]
susie.res.df.3$trans.pQTL.2 = chr.14.ids[3]

In [ ]:
chr.14.res <- rbind(susie.res.df.1, susie.res.df.2, susie.res.df.3)
chr.14.res$Locus = "14"

In [ ]:
chr.16.ids <- c("P10909-pQTL_1", "P19827-pQTL_1")

chr.16.loci <- lapply(chr.16.ids, function(id) {

    locus <- list()
    locus$beta <- trans.pqtl.loci[[id]]$pQTL_beta
    locus$varbeta <- trans.pqtl.loci[[id]]$pQTL_SE^2
    locus$snp <- trans.pqtl.loci[[id]]$SNP
    locus$position <- trans.pqtl.loci[[id]]$Position
    locus$type <- "quant"
    locus$sdY <- sd(protein.exp[,trans.pqtl.loci[[id]]$pQTL_Protein[1]], na.rm=TRUE)
    locus$LD <- cor(trans.pqtl.geno[, trans.pqtl.loci[[id]]$SNP], use="pairwise.complete.obs")
    
    locus.S = runsusie(locus)
    
    return(locus.S)
})

In [ ]:
susie.res = suppressMessages(coloc.susie(chr.16.loci[[1]], chr.16.loci[[2]]))

chr.16.res = data.frame(susie.res$summary)
chr.16.res$trans.pQTL.1 = chr.16.ids[1]
chr.16.res$trans.pQTL.2 = chr.16.ids[2]
chr.16.res$Locus = "16"

In [ ]:
coloc.res = dplyr::bind_rows(chr.14.res, chr.16.res)

## COLOC with cis-pQTL

In [ ]:
serpina1.id <- "P01009"

serpina1.locus <- list()
serpina1.locus$beta <- cis.pqtl.loci[[serpina1.id]]$pQTL_beta
serpina1.locus$varbeta <- cis.pqtl.loci[[serpina1.id]]$pQTL_SE^2
serpina1.locus$snp <- cis.pqtl.loci[[serpina1.id]]$SNP
serpina1.locus$position <- cis.pqtl.loci[[serpina1.id]]$Position
serpina1.locus$type <- "quant"
serpina1.locus$sdY <- sd(protein.exp[,cis.pqtl.loci[[serpina1.id]]$pQTL_Protein[1]], na.rm=TRUE)

abf.res = suppressWarnings(coloc.abf(chr.14.loci[[1]], serpina1.locus))

abf.res = suppressWarnings(coloc.abf(chr.14.loci[[2]], serpina1.locus))

abf.res = suppressWarnings(coloc.abf(chr.14.loci[[3]], serpina1.locus))

In [ ]:
hp.id <- "P00738"

hp.locus <- list()
hp.locus$beta <- cis.pqtl.loci[[hp.id]]$pQTL_beta
hp.locus$varbeta <- cis.pqtl.loci[[hp.id]]$pQTL_SE^2
hp.locus$snp <- cis.pqtl.loci[[hp.id]]$SNP
hp.locus$position <- cis.pqtl.loci[[hp.id]]$Position
hp.locus$type <- "quant"
hp.locus$sdY <- sd(protein.exp[,cis.pqtl.loci[[hp.id]]$pQTL_Protein[1]], na.rm=TRUE)

abf.res = suppressWarnings(coloc.abf(chr.16.loci[[1]], hp.locus))

abf.res = suppressWarnings(coloc.abf(chr.16.loci[[2]], hp.locus))

## Colocalizing Loci

In [ ]:
coloc.res %>%
    dplyr::filter(PP.H3.abf + PP.H4.abf > 0.25) %>%
    dplyr::mutate(Coloc.Factor=PP.H4.abf / (PP.H3.abf + PP.H4.abf)) %>%
    dplyr::filter(Coloc.Factor > 0.7) %>%
    dplyr::arrange(desc(Coloc.Factor))

In [ ]:
trans.pqtl.summary %>%
    dplyr::group_by(pQTL_ID) %>%
    dplyr::slice_min(pQTL_pval, n=1)

## Chr 14 Locus

The Chromosome 14 Locus for the overlapping trans-pQTL is in the intronic region of *SERPINA1*. The SERPIN family of proteins act as inhibitors of proteases involved in the immune response in blood. I will create association plots with LD information to visualize this locus.

In [ ]:
# DA Peaks
da.peaks <- read.csv("~/gains_team282/epigenetics/accessibility/analysis/atac_seq/immune/da_peak_set.csv", row.names=1)

In [ ]:
# SERPINA1
serpina1 = "ENSG00000197249"

serpina1.summary <- fread("/nfs/users/nfs_n/nm18/gains_team282/nikhil/colocalization/cis_eqtl/conditional_effects/LD/chr14_conditional_cis_eQTL_summary_statistics.tsv") %>%
    as.data.frame() %>%
    dplyr::filter(Gene == serpina1)

In [ ]:
gene.locus.data <- serpina1.summary %>%
    dplyr::mutate(Signal = as.character(Signal)) %>%
    dplyr::select(SNP, Position, P_Value, Signal) %>%
    dplyr::filter(Position < 94.5e6, Position > 94.25e6)

protein.locus.data <- trans.pqtl.summary %>%
    dplyr::filter(pQTL_ID %in% chr.14.ids) %>%
    dplyr::bind_rows(
        cis.pqtl.summary %>% 
            dplyr::filter(pQTL_Protein == serpina1.id) %>% 
            dplyr::select(SNP, Position, pQTL_pval) %>% 
            dplyr::mutate(pQTL_ID=serpina1.id)
    ) %>%
    dplyr::select(SNP, Position, P_Value=pQTL_pval, Signal=pQTL_ID) %>%
    dplyr::filter(Position < 94.5e6, Position > 94.25e6)

In [ ]:
# Load genotypes for the relevant loci
chr.geno <- fread("/nfs/users/nfs_n/nm18/gains_team282/nikhil/data/genotypes/eqtl_genotypes_14.raw", sep=" ", drop=2:6)
chr.geno <- as.data.frame(chr.geno)
colnames(chr.geno) <- gsub("X", "", colnames(chr.geno))
colnames(chr.geno) <- sapply(strsplit(colnames(chr.geno), "_"), function(x) { x[1] })
rownames(chr.geno) <- gsub("^GA", "", chr.geno[, 1])
chr.geno[, 1] <- NULL

gene.ld <- cor(chr.geno[,intersect(colnames(chr.geno), gene.locus.data$SNP)], use="pairwise.complete.obs")

In [ ]:
chr.geno <- fread("/nfs/users/nfs_n/nm18/gains_team282/nikhil/data/genotypes/trans_pqtl_genotypes.raw", sep=" ", drop=2:6)
chr.geno <- as.data.frame(chr.geno)
colnames(chr.geno) <- gsub("X", "", colnames(chr.geno))
colnames(chr.geno) <- sapply(strsplit(colnames(chr.geno), "_"), function(x) { x[1] })
rownames(chr.geno) <- gsub("^GA", "", chr.geno[, 1])
chr.geno[, 1] <- NULL

prot.ld <- cor(chr.geno[,intersect(colnames(chr.geno), protein.locus.data$SNP)], use="pairwise.complete.obs")

In [ ]:
gene.ld.info <- do.call(rbind, gene.locus.data %>%
    dplyr::group_by(Signal) %>%
    dplyr::slice_min(P_Value, n=1) %>%
    apply(., 1, function(x) {
        as.data.frame(gene.ld[, x[1]]) %>%
            dplyr::select(LD=1) %>%
            dplyr::mutate(SNP=rownames(.), Signal = x[4], LD=LD^2)
    })
)

protein.ld.info <- do.call(rbind, protein.locus.data %>%
    dplyr::group_by(Signal) %>%
    dplyr::slice_min(P_Value, n=1) %>%
    apply(., 1, function(x) {
        as.data.frame(prot.ld[, x[1]]) %>%
            dplyr::select(LD=1) %>%
            dplyr::mutate(SNP=rownames(.), Signal = x[4], LD=LD^2)
    })
)

In [ ]:
gene.locus.data %>%
    dplyr::group_by(Signal) %>%
    dplyr::slice_min(P_Value, n=1)

protein.locus.data %>%
    dplyr::group_by(Signal) %>%
    dplyr::slice_min(P_Value, n=1)

In [ ]:
gene.da.peaks <- da.peaks %>%
    dplyr::filter(Peak.Start >= min(gene.locus.data$Position), Peak.End <= max(gene.locus.data$Position), Peak.Chr == "14") %>%
    dplyr::mutate(Overlaps=ifelse(Peak.Start <= 94396493 & Peak.End >= 94396493, "eQTL 1", "None")) %>%
    dplyr::mutate(Overlaps=ifelse(Peak.Start <= 94375518 & Peak.End >= 94375518, "eQTL 2", Overlaps)) %>%
    dplyr::mutate(Overlaps=ifelse(Peak.Start <= 94309697 & Peak.End >= 94309697, "eQTL 3", Overlaps)) %>%
    dplyr::mutate(Overlaps=ifelse(Peak.Start <= 94344873 & Peak.End >= 94344873, "eQTL 4", Overlaps))

protein.da.peaks <- da.peaks %>%
    dplyr::filter(Peak.Start >= min(protein.locus.data$Position), Peak.End <= max(protein.locus.data$Position), Peak.Chr == "14") %>%
    dplyr::mutate(Overlaps=ifelse(Peak.Start <= 94380606 & Peak.End >= 94380606, "SERPINA1", "None")) %>%
    dplyr::mutate(Overlaps=ifelse(Peak.Start <= 94379979 & Peak.End >= 94379979, "CLU and ITIH1", Overlaps)) %>%
    dplyr::mutate(Overlaps=ifelse(Peak.Start <= 94378610 & Peak.End >= 94378610, "PRG4", Overlaps))

In [ ]:
options(repr.plot.width=14, repr.plot.height=8)

palette <- colorRampPalette(rev(RColorBrewer::brewer.pal(11, "Spectral")))

p1 <- merge(gene.locus.data, gene.ld.info, by=c("SNP", "Signal")) %>%
    dplyr::filter(!grepl("pQTL", Signal))%>%
    ggplot() +
    geom_point(aes(x=Position / 10^6, y=-log10(P_Value), fill=LD), pch=21, size=I(3)) +
    scale_fill_gradientn(colors=palette(100), limits=c(0, 1)) +
    guides(fill="none") +
    xlab("Position (Mb)") + ylab(bquote("-log"[10]*"(p-value)")) +
    facet_grid(Signal ~ ., scales="free_y") +
    ggplot_theme

p2 <- merge(protein.locus.data, protein.ld.info, by=c("SNP", "Signal")) %>%
    dplyr::filter(grepl("pQTL|P01009", Signal))%>%
    ggplot() +
    geom_point(aes(x=Position / 10^6, y=-log10(P_Value), fill=LD), pch=21, size=I(3)) +
    scale_fill_gradientn(colors=palette(100), limits=c(0, 1)) +
    xlab("Position (Mb)") + ylab(bquote("-log"[10]*"(p-value)")) +
    facet_grid(Signal ~ ., scales="free_y") +
    ggplot_theme +
    theme(legend.position="right")

p <- p1 + p2

ggsave(p, "01_Colocalization/results/trans_pqtl_chr_14.svg", width=14, height=8)

In [ ]:
cis.eqtl.signals <- split(serpina1.summary, serpina1.summary$Signal)

do.call(rbind, lapply(names(cis.eqtl.signals), function(eqtl.signal) {

    cis.eqtl.signals[[eqtl.signal]] <- cis.eqtl.signals[[eqtl.signal]] %>%
        dplyr::filter(!is.na(Beta))

    cis.eqtl.locus <- list()
    cis.eqtl.locus$beta <- cis.eqtl.signals[[eqtl.signal]]$Beta
    cis.eqtl.locus$varbeta <- cis.eqtl.signals[[eqtl.signal]]$SE^2
    cis.eqtl.locus$snp <- cis.eqtl.signals[[eqtl.signal]]$SNP
    cis.eqtl.locus$position <- cis.eqtl.signals[[eqtl.signal]]$Position
    cis.eqtl.locus$type <- "quant"
    cis.eqtl.locus$sdY <- sd(gene.exp[, serpina1], na.rm=TRUE)

    abf.res = suppressWarnings(coloc.abf(cis.eqtl.locus, chr.14.loci[[3]]))

    abf.res.df = data.frame(t(abf.res$summary))
    abf.res.df$eQTL.Signal = eqtl.signal

    return(abf.res.df)
}))

## Chr 16 Locus

In [ ]:
# DA Peaks
da.peaks <- read.csv("~/gains_team282/epigenetics/accessibility/analysis/atac_seq/immune/da_peak_set.csv", row.names=1)

In [ ]:
# HP
hp = "ENSG00000257017"

hp.summary <- fread("/nfs/users/nfs_n/nm18/gains_team282/nikhil/colocalization/cis_eqtl/conditional_effects/LD/chr16_conditional_cis_eQTL_summary_statistics.tsv") %>%
    as.data.frame() %>%
    dplyr::filter(Gene == hp)

In [ ]:
gene.locus.data <- hp.summary %>%
    dplyr::mutate(Signal = as.character(Signal)) %>%
    dplyr::select(SNP, Position, P_Value, Signal) %>%
    dplyr::filter(Position < 72.5e6, Position > 71.75e6)

protein.locus.data <- trans.pqtl.summary %>%
    dplyr::filter(pQTL_ID %in% chr.16.ids) %>%
    dplyr::bind_rows(
        cis.pqtl.summary %>% 
            dplyr::filter(pQTL_Protein == hp.id) %>% 
            dplyr::select(SNP, Position, pQTL_pval) %>% 
            dplyr::mutate(pQTL_ID=hp.id)
    ) %>%
    dplyr::select(SNP, Position, P_Value=pQTL_pval, Signal=pQTL_ID) %>%
    dplyr::filter(Position < 72.5e6, Position > 71.75e6)

In [ ]:
# Load genotypes for the relevant loci
chr.geno <- fread("/nfs/users/nfs_n/nm18/gains_team282/nikhil/data/genotypes/eqtl_genotypes_16.raw", sep=" ", drop=2:6)
chr.geno <- as.data.frame(chr.geno)
colnames(chr.geno) <- gsub("X", "", colnames(chr.geno))
colnames(chr.geno) <- sapply(strsplit(colnames(chr.geno), "_"), function(x) { x[1] })
rownames(chr.geno) <- gsub("^GA", "", chr.geno[, 1])
chr.geno[, 1] <- NULL

gene.ld <- cor(chr.geno[,intersect(colnames(chr.geno), gene.locus.data$SNP)], use="pairwise.complete.obs")

In [ ]:
chr.geno <- fread("/nfs/users/nfs_n/nm18/gains_team282/nikhil/data/genotypes/trans_pqtl_genotypes.raw", sep=" ", drop=2:6)
chr.geno <- as.data.frame(chr.geno)
colnames(chr.geno) <- gsub("X", "", colnames(chr.geno))
colnames(chr.geno) <- sapply(strsplit(colnames(chr.geno), "_"), function(x) { x[1] })
rownames(chr.geno) <- gsub("^GA", "", chr.geno[, 1])
chr.geno[, 1] <- NULL

prot.ld <- cor(chr.geno[,intersect(colnames(chr.geno), protein.locus.data$SNP)], use="pairwise.complete.obs")

In [ ]:
gene.ld.info <- do.call(rbind, gene.locus.data %>%
    dplyr::group_by(Signal) %>%
    dplyr::slice_min(P_Value, n=1) %>%
    apply(., 1, function(x) {
        as.data.frame(gene.ld[, x[1]]) %>%
            dplyr::select(LD=1) %>%
            dplyr::mutate(SNP=rownames(.), Signal = x[4], LD=LD^2)
    })
)

protein.ld.info <- do.call(rbind, protein.locus.data %>%
    dplyr::group_by(Signal) %>%
    dplyr::slice_min(P_Value, n=1) %>%
    apply(., 1, function(x) {
        as.data.frame(prot.ld[, x[1]]) %>%
            dplyr::select(LD=1) %>%
            dplyr::mutate(SNP=rownames(.), Signal = x[4], LD=LD^2)
    })
)

In [ ]:
gene.locus.data %>%
    dplyr::group_by(Signal) %>%
    dplyr::slice_min(P_Value, n=1)

protein.locus.data %>%
    dplyr::group_by(Signal) %>%
    dplyr::slice_min(P_Value, n=1)

In [ ]:
gene.da.peaks <- da.peaks %>%
    dplyr::filter(Peak.Start >= min(gene.locus.data$Position), Peak.End <= max(gene.locus.data$Position), Peak.Chr == "16") %>%
    dplyr::mutate(Overlaps=ifelse(Peak.Start <= 72001460 & Peak.End >= 72001460, "eQTL", "None"))

protein.da.peaks <- da.peaks %>%
    dplyr::filter(Peak.Start >= min(protein.locus.data$Position), Peak.End <= max(protein.locus.data$Position), Peak.Chr == "16") %>%
    dplyr::mutate(Overlaps=ifelse(Peak.Start <= 72008529 & Peak.End >= 72008529, "HP", "None")) %>%
    dplyr::mutate(Overlaps=ifelse(Peak.Start <= 72110275 & Peak.End >= 72110275, "CLU and ITIH1", Overlaps))

In [ ]:
options(repr.plot.width=14, repr.plot.height=8)

palette <- colorRampPalette(rev(RColorBrewer::brewer.pal(11, "Spectral")))

p1 <- merge(gene.locus.data, gene.ld.info, by=c("SNP", "Signal")) %>%
    dplyr::filter(!grepl("pQTL", Signal))%>%
    ggplot() +
    geom_point(aes(x=Position / 10^6, y=-log10(P_Value), fill=LD), pch=21, size=I(3)) +
    scale_fill_gradientn(colors=palette(100), limits=c(0, 1)) +
    guides(fill="none") +
    xlab("Position (Mb)") + ylab(bquote("-log"[10]*"(p-value)")) +
    facet_grid(Signal ~ ., scales="free_y") +
    ggplot_theme

p2 <- merge(protein.locus.data, protein.ld.info, by=c("SNP", "Signal")) %>%
    dplyr::filter(grepl("pQTL|P00738", Signal))%>%
    ggplot() +
    geom_point(aes(x=Position / 10^6, y=-log10(P_Value), fill=LD), pch=21, size=I(3)) +
    scale_fill_gradientn(colors=palette(100), limits=c(0, 1)) +
    xlab("Position (Mb)") + ylab(bquote("-log"[10]*"(p-value)")) +
    facet_grid(Signal ~ ., scales="free_y") +
    ggplot_theme +
    theme(legend.position="right")

p <- p1 + p2

ggsave(p, "01_Colocalization/results/trans_pqtl_chr_16.svg", width=14, height=8)

In [ ]:
cis.eqtl.signals <- split(hp.summary, hp.summary$Signal)

do.call(rbind, lapply(names(cis.eqtl.signals), function(eqtl.signal) {

    cis.eqtl.signals[[eqtl.signal]] <- cis.eqtl.signals[[eqtl.signal]] %>%
        dplyr::filter(!is.na(Beta))

    cis.eqtl.locus <- list()
    cis.eqtl.locus$beta <- cis.eqtl.signals[[eqtl.signal]]$Beta
    cis.eqtl.locus$varbeta <- cis.eqtl.signals[[eqtl.signal]]$SE^2
    cis.eqtl.locus$snp <- cis.eqtl.signals[[eqtl.signal]]$SNP
    cis.eqtl.locus$position <- cis.eqtl.signals[[eqtl.signal]]$Position
    cis.eqtl.locus$type <- "quant"
    cis.eqtl.locus$sdY <- sd(gene.exp[, hp], na.rm=TRUE)

    abf.res = suppressWarnings(coloc.abf(cis.eqtl.locus, chr.16.loci[[1]]))

    abf.res.df = data.frame(t(abf.res$summary))
    abf.res.df$eQTL.Signal = eqtl.signal

    return(abf.res.df)
}))